In [15]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from PyQt5.QtWidgets import (QSystemTrayIcon, QApplication, QLabel, QMainWindow, QPushButton, QWidget, 
                             QVBoxLayout, QTextEdit, QDialog, QComboBox)
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import Qt, QCoreApplication
import tkinter as tk
import PIL.Image
import numpy as np
import cv2
import io
import json
import ctypes
from io import BytesIO
import win32clipboard
import glob
import os
from matplotlib import pyplot as plt
from IPython.display import clear_output, Image, display
from google.cloud.vision import types
from google.cloud import vision
import mss
import pyperclip
import mss.tools
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/poorv/Downloads/ocr_scripts/keys/direct-outlook-270501-c05c2d97d1c6.json"

In [33]:
def send_text_clipboard(text):
    pyperclip.copy(text)
    spam = pyperclip.paste()
    
def dtlh(path, lh):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()
    if type(path) == str:
        with io.open(path, 'rb') as image_file:
            content = image_file.read()
            image = vision.types.Image(content=content)
    else:
        image = types.Image(content=cv2.imencode('.jpg', path)[1].tostring())
        
    response = client.text_detection(
    image=image,
    image_context={"language_hints": lh},
    )
    texts = response.text_annotations
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    try:
        return texts[0].description
    except IndexError:
        print('no text in image')

In [35]:
dtlh(r"C:\Users\poorv\Downloads\ocr_scripts\demo\pg191_excerpt.png", [])


"ఎటితే\nఎటికే exata\n?\nn. [move] pulling (C. (Kitt.)) [Ka.\nD504(a)].\n? wes eravu\nn. [move] 1 pulling, dragging\n2 spasm, cramp (My. (Kitt.)) [Ka. D504(a)].\nఎటివు' eravu\n2\nw vt. [move] 1 to rub;\nto rub off or out; to smear gently 2 to stroke gently\n[Ka. D505].\n"

In [39]:
class ocrWidget(QtWidgets.QWidget):
#     clicked = QtCore.pyqtSignal(object)
    
    def __init__(self, parent = None, lang_hint = []):
        super().__init__()
        root = tk.Tk()
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.setGeometry(0, 0, screen_width, screen_height)
        self.setWindowTitle(' ')
        self.begin = QtCore.QPoint()
        self.end = QtCore.QPoint()
        self.parent = parent
        self.setWindowOpacity(0.3)
        QtWidgets.QApplication.setOverrideCursor(
            QtGui.QCursor(QtCore.Qt.CrossCursor)
        )
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)
        QtWidgets.QShortcut(
            QtGui.QKeySequence("Escape"), self, activated=self.on_Escape
        )
        self.lang_hint = lang_hint
        print(self.lang_hint)

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.setPen(QtGui.QPen(QtGui.QColor('black'), 3))
        qp.setBrush(QtGui.QColor(128, 128, 255, 128))
        qp.drawRect(QtCore.QRect(self.begin, self.end))

    def mousePressEvent(self, event):
        self.begin = event.pos()
        self.end = self.begin
        self.update()
    
    @QtCore.pyqtSlot()
    def on_Escape(self):
        print("main esp exit")
#         self.close()
        self.closeAndReturn()
        

    def mouseMoveEvent(self, event):
        self.end = event.pos()
        self.update()
    
    def closeAndReturn(self):
        self.close()
        self.parent.setWindowOpacity(1.)
        return

    def mouseReleaseEvent(self, event):
        previous = self.parent
        self.close()

        x1 = min(self.begin.x(), self.end.x())
        y1 = min(self.begin.y(), self.end.y())
        x2 = max(self.begin.x(), self.end.x())
        y2 = max(self.begin.y(), self.end.y())
        
        if (y1 == y2 and x1 == x2):
            print("no region selected")
            self.closeAndReturn()
        
        with mss.mss() as sct:
            # The screen part to capture            
            monitor = {"top": y1, "left":x1, "width": abs(x2 - x1) , "height": abs(y2 - y1)}
            output = "sct-{top}x{left}_{width}x{height}.png".format(**monitor)
            sct_img = sct.grab(monitor)
            try:
                img = PIL.Image.frombytes("RGB", sct_img.size, sct_img.bgra, "raw", "BGRX")
                img = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
                im_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                ret,thresh = cv2.threshold(im_gray,127,255,cv2.THRESH_BINARY_INV)
#                 ocr = dtlh(thresh, ["hi"])
                ocr = dtlh(thresh, self.lang_hint)
                if ocr:
#                     send_text_clipboard(ocr)
                    print(ocr)
                self.parent.setWindowOpacity(1.)
            except ValueError:
                print("invalid region selection. try again")
                self.closeAndReturn()

In [44]:
lang_hints = {u'Afrikaans - Afrikaans': "af", 
u'shqip - Albanian': "sq", 
u'العربية - Arabic': "ar", 
u'Հայ - Armenian': "hy", 
u'беларускі - Belorussian': "be", 
u'বাংলা - Bengali': "bn", 
u'български - Bulgarian': "bg", 
u'Català - Catalan': "ca", 
u'普通话 - Chinese': "zh", 
u'Hrvatski - Croatian': "hr", 
u'Čeština - Czech': "cs", 
u'Dansk - Danish': "da", 
u'Nederlands - Dutch': "nl", 
u'English - English ': "en", 
u'Eesti keel - Estonian': "et", 
u'Filipino - Filipino': "fil", 
u'Suomi - Finnish': "fi", 
u'Français - French ': "fr", 
u'Deutsch - German': "de", 
u'Ελληνικά - Greek': "el", 
u'ગુજરાતી - Gujarati': "gu", 
u'עברית - Hebrew': "iw", 
u'हिन्दी - Hindi': "hi", 
u'Magyar - Hungarian': "hu", 
u'Íslenska - Icelandic': "is", 
u'Bahasa Indonesia - Indonesian': "id", 
u'Italiano - Italian': "it", 
u'日本語 - Japanese': "ja", 
u'ಕನ್ನಡ - Kannada': "kn", 
u'ភាសាខ្មែរ - Khmer': "km", 
u'한국어 - Korean': "ko", 
u'ລາວ - Lao': "lo", 
u'Latviešu - Latvian': "lv", 
u'Lietuvių - Lithuanian': "lt", 
u'Македонски - Macedonian': "mk", 
u'Bahasa Melayu - Malay': "ms", 
u'മലയാളം - Malayalam': "ml", 
u'मराठी - Marathi': "mr", 
u'नेपाली - Nepali': "ne", 
u'Norsk - Norwegian': "no", 
u'فارسی - Persian': "fa", 
u'Polski - Polish': "pl", 
u'Português - Portuguese': "pt", 
u'ਪੰਜਾਬੀ - Punjabi': "pa", 
u'Română - Romanian': "ro", 
u'Русский - Russian': "ru", 
u'Русский (старая орфография) - Russian': "ru-PETR1708", 
u'Српски - Serbian': "sr", 
u'Српски (латиница) - Serbian': "sr-Latn", 
u'Slovenčina - Slovak': "sk", 
u'Slovenščina - Slovenian': "sl", 
u'Español - Spanish': "es", 
u'Svenska - Swedish': "sv", 
u'தமிழ் - Tamil': "ta", 
u'తెలుగు - Telugu': "te", 
u'ไทย - Thai': "th", 
u'Türkçe - Turkish': "tr", 
u'Українська - Ukrainian': "uk", 
u'Tiếng Việt - Vietnamese': "vi", 
u'Yiddish - Yiddish': "yi"}

In [43]:
from PyQt5 import QtGui, QtCore
import sys


class First(QMainWindow):
    def __init__(self, parent=None):
        super(First, self).__init__(parent)
        self.pushButton = QPushButton("click me")
        self.setCentralWidget(self.pushButton)
        self.pushButton.clicked.connect(self.on_pushButton_clicked)
        self.dialogs = list()
        self.combo = QComboBox(self)
        self.combo.addItem("Kannada", "kn")
        self.combo.addItem("Hindi", "hi")
        self.combo.activated.connect(self.handleActivated)
        self.lang_hint = []
    
    def on_pushButton_clicked(self):
        dialog = ocrWidget(self, self.lang_hint)
        self.dialogs.append(dialog)
        dialog.show()
        self.setWindowOpacity(0.)
        
    def handleActivated(self, index):
        self.lang_hint = [self.combo.itemData(index)]


def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    main = First()
    main.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

['hi']
ndleActivated (self, index):
f.lang_hint = [self.combo. itemData(index)]
print("lang_hint", self.lang_hint)
tCore. QCoreApplication.instance()

['hi']
.
बारिश होने लगी।
बच्चे तालियाँ बजाने लगे ।
माँ मेरा नाम चिल्लाने लगी।
हमें प्यास लगने लगी।
मुझे महसूस होने लगा कि यह पाट
तो कभी खत्म नहीं होगा।



SystemExit: 0

In [ ]:
#a button to select the ocr languge 
#upon selecting language, pass in value to ocr widget
#somehow remeber the favorite langs selected
#create keyboard shorcuts??
#consider logic for setting multuple lang hints (i.e. checkbox)